In [1]:
#爬取--新浪财经/资产负债表
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 设置Selenium的Chrome浏览器选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式，不显示浏览器窗口
chrome_options.add_argument("--disable-gpu")  # 禁用GPU加速
chrome_options.add_argument("--no-sandbox")

# 指定chromedriver的路径（需提前下载对应版本的chromedriver并指定路径）
driver_path = 'path_to_chromedriver'

# 创建Chrome浏览器实例
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

# 初始化结果字典
result = {}

# 循环迭代年份
for year in range(2017, 2023):
    # 构建URL
    url = f"https://money.finance.sina.com.cn/corp/go.php/vFD_BalanceSheet/stockid/603486/ctrl/{year}/displaytype/4.phtml"

    # 访问目标网页
    driver.get(url)

    # 获取网页源代码
    html = driver.page_source

    # 使用BeautifulSoup解析网页内容
    soup = BeautifulSoup(html, 'html.parser')

    # 查找指定表格
    table_id = 'BalanceSheetNewTable0'
    table = soup.find('table', {'id': table_id})

    # 初始化要提取的表格行的标题
    desired_rows = ["流动资产合计", "非流动资产合计", "资产总计",
                    "流动负债合计", "非流动负债合计", "负债合计",
                    "所有者权益(或股东权益)合计", "负债和所有者权益(或股东权益)总计"]

     # 查找并提取指定表格行的数据
    rows = table.find_all('tr')
    df_year = pd.DataFrame()
    for row in rows:
        cells = row.find_all('td')
        if cells and cells[0].text.strip() in desired_rows:
            row_data = [cell.text.strip().replace(',', '') for cell in cells[1:]]
            df_year[cells[0].text.strip()] = row_data
    result[str(year)] = df_year
    # print(df_year)
# 关闭浏览器
driver.quit()

C:\Users\belle\AppData\Local\Temp\ipykernel_3440\4180044449.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)


In [5]:
# 对每个年份的DataFrame进行求和
sum_data = {}
for year, df in result.items():
    print(year)
    filename = f"data\df_sum_{year}.xlsx"
    # 对合并后的DataFrame进行求和，并将字符串转换为浮点数类型
    df_sum = df.apply(lambda x: pd.to_numeric(x, errors='coerce')).sum()
    df_sum.to_excel(filename)
    print(df_sum)

2017
流动资产合计               372059.34
非流动资产合计              119838.81
资产总计                 491898.15
流动负债合计               259252.33
非流动负债合计                 936.91
负债合计                 260189.24
所有者权益(或股东权益)合计       231708.91
负债和所有者权益(或股东权益)总计    491898.15
dtype: float64
2018
流动资产合计                990543.69
非流动资产合计               229010.49
资产总计                 1219554.17
流动负债合计                514464.02
非流动负债合计                 3407.25
负债合计                  517871.27
所有者权益(或股东权益)合计        701682.91
负债和所有者权益(或股东权益)总计    1219554.17
dtype: float64
2019
流动资产合计               1208332.34
非流动资产合计               424729.36
资产总计                 1633061.71
流动负债合计                622405.56
非流动负债合计                 6105.76
负债合计                  628511.32
所有者权益(或股东权益)合计       1004550.40
负债和所有者权益(或股东权益)总计    1633061.71
dtype: float64
2020
流动资产合计               1484318.53
非流动资产合计               459017.46
资产总计                 1943335.99
流动负债合计                841739.63
非流动负债合计                 4088.05
负债合计           

In [7]:
#爬取新浪财经利润表
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

base_url = "https://money.finance.sina.com.cn/corp/go.php/vFD_ProfitStatement/stockid/603486/ctrl/{}/displaytype/4.phtml"

# 创建一个Excel工作簿
workbook = Workbook()

# 遍历2017年至2022年的数据
for year in range(2013, 2023):
    url = base_url.format(year)

    # 发送GET请求获取网页内容
    response = requests.get(url)

    # 解析网页内容
    soup = BeautifulSoup(response.text, "html.parser")

    # 找到目标表格
    table = soup.find("table", id="ProfitStatementNewTable0")

    # 提取表格中的数据
    rows = table.find_all("tr")

    # 创建一个工作表，以年份命名
    sheet = workbook.create_sheet(str(year))

    # 遍历表格的行和列，将数据写入工作表
    for row in rows:
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]

        # 判断是否为目标行数据
        if len(row_data) > 0 and ("营业收入" in row_data[0] or "营业利润" in row_data[0] or "报表日期" in row_data[0]):
            sheet.append(row_data)

# 删除默认创建的工作表
workbook.remove(workbook["Sheet"])

# 保存Excel文件
workbook.save("data\data_2013_2022.xlsx")